In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_10%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,NaN,0.500,6.68,NaN,0.2,...,0.4,NaN,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1.0
1,3.89,0.00,NaN,0.000,4.83,NaN,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0.0,NaN,0.0,NaN,1.416,NaN,NaN,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,NaN,0.2,NaN,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,NaN,2.0,1.578,0.4,0.5,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,NaN,0.074,NaN,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,NaN,0.0
773,55.57,0.24,0.2,NaN,NaN,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,NaN,0.040,0.00,0.2,0.000,0.20,5.0,NaN,...,0.4,NaN,0.2,0.0,2.0,NaN,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      74
PARA_A            62
Score_A           76
Risk_A            87
PARA_B            77
Score_B           83
Risk_B            93
TOTAL             94
numbers           88
Score_B.1         94
Risk_C            91
Money_Value       71
Score_MV          84
Risk_D            71
District_Loss     80
PROB              63
RiSk_E            93
History           73
Prob              89
Risk_F            80
Score             75
Inherent_Risk     71
CONTROL_RISK      92
Detection_Risk    75
Audit_Risk        81
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 2017
persentase : 9.997026169706581


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score       9.536082
PARA_A             7.989691
Score_A            9.793814
Risk_A            11.211340
PARA_B             9.922680
Score_B           10.695876
Risk_B            11.984536
TOTAL             12.113402
numbers           11.340206
Score_B.1         12.113402
Risk_C            11.726804
Money_Value        9.149485
Score_MV          10.824742
Risk_D             9.149485
District_Loss     10.309278
PROB               8.118557
RiSk_E            11.984536
History            9.407216
Prob              11.469072
Risk_F            10.309278
Score              9.664948
Inherent_Risk      9.149485
CONTROL_RISK      11.855670
Detection_Risk     9.664948
Audit_Risk        10.438144
Risk               0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,9)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.89,4.18,0.6,2.508,2.50,0.0,0.500,6.68,0.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148
1,3.89,0.00,0.0,0.000,4.83,0.0,0.966,4.83,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.0,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.2,0.4,0.0,0.0,0.0,0.0,1.416,0.0,0.0,0.2832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.00,0.2,0.000,0.40,0.2,0.080,0.89,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.5,0.3156
772,55.57,0.47,0.2,0.094,0.37,0.0,0.074,0.00,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.0000
773,55.57,0.24,0.2,0.000,0.00,0.2,0.008,0.28,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912
774,55.57,0.20,0.0,0.040,0.00,0.2,0.000,0.20,5.0,0.0,...,0.2,0.4,0.0,0.2,0.0,2.0,0.000,0.4,0.5,0.2880


In [13]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4780


In [14]:
euc_dist = {}

In [15]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)

Kolom : 1
[2.72, 3.41, 3.41, 3.89, 3.89, 3.89, 3.89, 3.89, 3.89]
min :  2.72
max :  3.89
mean :  3.6533333333333338
fuzzy :  [(2.72, 0.0), (3.89, 0.0), (3.89, 0.0), (3.89, 0.0), (3.41, 0.739285714285714), (3.89, 0.0), (3.89, 0.0), (3.41, 0.739285714285714), (3.89, 0.0)]
fuzzy :  None
pembilang :  5.04192857142857
penyebut :  1.478571428571428
rata-rata tertimbang :  3.41
 
[1.99, 2.72, 3.89, 55.57, 55.57, 55.57, 55.57, 59.85, 59.85]
min :  1.99
max :  59.85
mean :  38.95333333333333
fuzzy :  [(55.57, 0.20481735524007022), (55.57, 0.20481735524007022), (59.85, 0.0), (3.89, 0.05140229055821085), (2.72, 0.019749301109207328), (55.57, 0.20481735524007022), (1.99, 0.0), (59.85, 0.0), (55.57, 0.20481735524007022)]
fuzzy :  None
pembilang :  45.78047473205129
penyebut :  0.890421012627699
rata-rata tertimbang :  51.41441417352639
 
[1.85, 1.99, 2.37, 2.37, 2.37, 2.37, 3.41, 55.57, 55.57]
min :  1.85
max :  55.57
mean :  14.207777777777778
fuzzy :  [(2.37, 0.042078762812443805), (2.37, 0.04207

[0.0, 2.72, 2.72, 2.72, 2.72, 2.72, 2.72, 3.41, 3.41]
min :  0.0
max :  3.41
mean :  2.571111111111111
fuzzy :  [(2.72, 0.8225165562913905), (2.72, 0.8225165562913905), (2.72, 0.8225165562913905), (0.0, 0.0), (2.72, 0.8225165562913905), (2.72, 0.8225165562913905), (3.41, 0.0), (2.72, 0.8225165562913905), (3.41, 0.0)]
fuzzy :  None
pembilang :  13.423470198675496
penyebut :  4.935099337748342
rata-rata tertimbang :  2.720000000000001
 
[2.72, 2.72, 2.72, 2.72, 2.72, 2.72, 2.720000000000001, 3.41, 3.41]
min :  2.72
max :  3.41
mean :  2.8733333333333335
fuzzy :  [(2.720000000000001, 5.7924679545660346e-15), (2.72, 0.0), (2.72, 0.0), (2.72, 0.0), (2.72, 0.0), (3.41, 0.0), (2.72, 0.0), (3.41, 0.0), (2.72, 0.0)]
fuzzy :  None
pembilang :  1.575551283641962e-14
penyebut :  5.7924679545660346e-15
rata-rata tertimbang :  2.720000000000001
 
[2.72, 2.72, 3.41, 3.41, 3.89, 3.89, 3.89, 55.57, 59.85]
min :  2.72
max :  59.85
mean :  15.483333333333333
fuzzy :  [(2.72, 0.0), (3.89, 0.09166884303995

[0.0, 1.85, 1.99, 1.99, 2.37, 2.72, 3.89, 3.89, 55.57]
min :  0.0
max :  55.57
mean :  8.252222222222223
fuzzy :  [(0.0, 0.0), (2.37, 0.2871953682509762), (2.72, 0.3296081863471119), (3.89, 0.47138817826847984), (1.85, 0.22418203850814594), (1.99, 0.24114716574660022), (1.99, 0.24114716574660022), (3.89, 0.47138817826847984), (55.57, 0.0)]
fuzzy :  None
pembilang :  6.61908980745927
penyebut :  2.266056281136394
rata-rata tertimbang :  2.920973262032086
 
[0.0, 1.85, 1.85, 1.85, 2.37, 2.72, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  19.705555555555556
fuzzy :  [(0.0, 0.0), (1.85, 0.09388215393290104), (2.72, 0.13803213983648155), (55.57, 0.0), (55.57, 0.0), (2.37, 0.12027065125458133), (1.85, 0.09388215393290104), (1.85, 0.09388215393290104), (55.57, 0.0)]
fuzzy :  None
pembilang :  1.1815348181561882
penyebut :  0.539949252889766
rata-rata tertimbang :  2.1882330827067666
 
[0.0, 1.99, 1.99, 1.99, 1.99, 3.41, 3.89, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  14.04444444444

[0.0, 3.89, 21.61, 55.57, 59.85, 59.85, 59.85, 59.85, 59.85]
min :  0.0
max :  59.85
mean :  42.257777777777775
fuzzy :  [(0.0, 0.0), (59.85, 0.0), (59.85, 0.0), (59.85, 0.0), (59.85, 0.0), (3.89, 0.09205405973916703), (21.61, 0.5113851493479176), (59.85, 0.0), (55.57, 0.24328933240699804)]
fuzzy :  None
pembilang :  24.928711571650737
penyebut :  0.8467285414940826
rata-rata tertimbang :  29.44120854561385
 
[0.0, 1.85, 1.99, 3.89, 3.89, 55.57, 59.85, 59.85, 59.85]
min :  0.0
max :  59.85
mean :  27.415555555555553
fuzzy :  [(0.0, 0.0), (59.85, 0.0), (59.85, 0.0), (1.99, 0.07258652832941559), (3.89, 0.141890248844938), (55.57, 0.13195848035353364), (1.85, 0.06747993839669289), (3.89, 0.141890248844938), (59.85, 0.0)]
fuzzy :  None
pembilang :  8.706123966668901
penyebut :  0.5558054447695182
rata-rata tertimbang :  15.663977473770814
 
[0.0, 0.0, 2.37, 2.37, 2.37, 3.89, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  19.745555555555555
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (2.37, 0.1

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53, 1.01]
min :  0.0
max :  1.01
mean :  0.1711111111111111
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.01, 0.0), (0.53, 0.5721854304635762), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  None
pembilang :  0.3032582781456954
penyebut :  0.5721854304635762
rata-rata tertimbang :  0.53
 
[0.0, 0.0, 0.0, 0.0, 0.11, 0.23, 0.32, 0.83, 1.19]
min :  0.0
max :  1.19
mean :  0.29777777777777775
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.23, 0.7723880597014926), (0.0, 0.0), (1.19, 0.0), (0.32, 0.9750933997509338), (0.83, 0.4034869240348692), (0.0, 0.0), (0.11, 0.3694029850746269)]
fuzzy :  None
pembilang :  0.8652076169587926
penyebut :  2.5203713685619222
rata-rata tertimbang :  0.3432857664354695
 
[0.0, 2.86, 5.22, 6.31, 6.67, 8.5, 10.08, 12.28, 14.04]
min :  0.0
max :  14.04
mean :  7.32888888888889
fuzzy :  [(6.31, 0.8609763493026075), (10.08, 0.590066225165563), (12.28, 0.2622516556291391), (0.0, 0.0), (14.04, 0.0), (2.86, 0.39023650697392354

...

[0.0, 12.7968, 14.7472, 18.5712, 21.4896, 33.890360145893844, 45.3568, 55.952, 56.7096]
min :  0.0
max :  56.7096
mean :  28.83484001621043
fuzzy :  [(18.5712, 0.6440542062851609), (21.4896, 0.7452651024912548), (45.3568, 0.40727884317576785), (33.890360145893844, 0.8186344875212046), (56.7096, 0.0), (55.952, 0.02717870935715974), (12.7968, 0.44379646264053724), (14.7472, 0.5114368587344126), (0.0, 0.0)]
fuzzy :  None
pembilang :  88.93513042855494
penyebut :  3.5976446702054976
rata-rata tertimbang :  24.720376407677573
 
[0.0, 1.0744, 1.6872, 1.922, 2.8984, 3.2472, 6.4692, 8.58, 9.3468]
min :  0.0
max :  9.3468
mean :  3.913911111111111
fuzzy :  [(1.922, 0.4910688938600774), (6.4692, 0.5296629581151833), (1.6872, 0.43107775115542285), (9.3468, 0.0), (8.58, 0.14114037958115183), (3.2472, 0.8296560416974211), (0.0, 0.0), (2.8984, 0.7405380239146975), (1.0744, 0.2745080226655917)]
fuzzy :  None
pembilang :  11.44399478785966
penyebut :  3.4376520709895453
rata-rata tertimbang :  3.32901

[18.5712, 18.6056, 21.4896, 22.9984, 24.720376407677573, 33.890360145893844, 39.0624, 47.3616, 56.7096]
min :  18.5712
max :  56.7096
mean :  31.489904061507932
fuzzy :  [(47.3616, 0.37066267661587565), (22.9984, 0.34269691285762266), (33.890360145893844, 0.90481819882998), (56.7096, 0.0), (39.0624, 0.699738809026068), (24.720376407677573, 0.4759901905330752), (18.5712, 0.0), (18.6056, 0.002662805791990769), (21.4896, 0.22590501230657878)]
fuzzy :  None
pembilang :  100.1055581123288
penyebut :  3.022474605961191
rata-rata tertimbang :  33.12039674870776
 
[3.636, 3.7716, 3.982, 3.984, 6.693546406912872, 8.376, 10.8756, 11.1312, 12.2376]
min :  3.636
max :  12.2376
mean :  7.1875051563236525
fuzzy :  [(3.7716, 0.0381809948265333), (3.982, 0.09742348237448785), (3.636, 0.0), (3.984, 0.09798662389110332), (11.1312, 0.21908499429182365), (8.376, 0.7646589063243907), (10.8756, 0.26969790512062874), (6.693546406912872, 0.8609156603556496), (12.2376, 0.0)]
fuzzy :  None
pembilang :  18.46148

[0.0, 0.3888, 0.4036, 0.4124, 0.4152, 0.4164, 0.4312, 0.4428, 0.4504]
min :  0.0
max :  0.4504
mean :  0.3734222222222222
fuzzy :  [(0.0, 0.0), (0.4504, 0.0), (0.4164, 0.44168591224018494), (0.4152, 0.4572748267898383), (0.4428, 0.0987297921478059), (0.4036, 0.6079676674364894), (0.4124, 0.49364896073903025), (0.3888, 0.8002309468822173), (0.4312, 0.2494226327944571)]
fuzzy :  None
pembilang :  1.2851334872979217
penyebut :  3.1489607390300227
rata-rata tertimbang :  0.40811353135313544
 
[0.0, 0.0, 0.28, 0.28286808510638306, 0.294, 0.298, 0.3032, 0.304, 0.314]
min :  0.0
max :  0.314
mean :  0.230674231678487
fuzzy :  [(0.0, 0.0), (0.28, 0.40803704166051924), (0.3032, 0.1296117661745177), (0.0, 0.0), (0.304, 0.12001089460603526), (0.314, 0.0), (0.294, 0.2400217892120705), (0.298, 0.1920174313696564), (0.28286808510638306, 0.3736168957181919)]
fuzzy :  None
pembilang :  0.42350386756099634
penyebut :  1.4633158187409911
rata-rata tertimbang :  0.2894138518405213
 
[0.0, 1.2868, 1.31, 1

[0.0, 0.28, 0.28, 0.2824, 0.2824629921259842, 0.286, 0.288, 0.292, 0.2932]
min :  0.0
max :  0.2932
mean :  0.25378477690288714
fuzzy :  [(0.0, 0.0), (0.28, 0.3348959859361262), (0.28, 0.3348959859361262), (0.2824629921259842, 0.2724076392403396), (0.286, 0.18267053778334272), (0.288, 0.13192872173241443), (0.2824, 0.2740058066750134), (0.2932, 0.0), (0.292, 0.030445089630557823)]
fuzzy :  None
pembilang :  0.4409952806241507
penyebut :  1.5612497669339205
rata-rata tertimbang :  0.28246299212598425
 
[0.0, 13.3564, 13.8436, 14.1292, 16.8476, 20.770055694360522, 23.714195974099383, 32.436, 57.3328]
min :  0.0
max :  57.3328
mean :  21.381094629828876
fuzzy :  [(0.0, 0.0), (14.1292, 0.6608267838770182), (57.3328, 0.0), (13.8436, 0.6474691890043236), (23.714195974099383, 0.9351045709724172), (13.3564, 0.6246827036332564), (32.436, 0.6925067877491201), (20.770055694360522, 0.9714215316826721), (16.8476, 0.7879671406765033)]
fuzzy :  None
pembilang :  104.73300805722069
penyebut :  5.31997

[0.0, 0.9088, 0.936, 0.97, 0.9932, 1.2348, 1.2668, 1.316, 2.7048]
min :  0.0
max :  2.7048
mean :  1.1478222222222223
fuzzy :  [(0.0, 0.0), (1.316, 0.8919844713404887), (2.7048, 0.0), (1.2348, 0.9441367892212835), (0.9088, 0.7917602416169751), (1.2668, 0.923584151632793), (0.936, 0.8154572911019903), (0.97, 0.845078602958259), (0.9932, 0.8652907922248896)]
fuzzy :  None
pembilang :  6.671620866663188
penyebut :  6.07729234009668
rata-rata tertimbang :  1.097794954283383
 
[0.0, 0.0, 0.3696, 0.3808, 0.3824, 0.3832, 0.3912, 0.3944, 0.3992]
min :  0.0
max :  0.3992
mean :  0.3000888888888889
fuzzy :  [(0.0, 0.0), (0.3944, 0.048430493273542874), (0.3808, 0.18565022421524638), (0.3992, 0.0), (0.3824, 0.16950672645739895), (0.3832, 0.1614349775784755), (0.3912, 0.08071748878923775), (0.3696, 0.29865470852017956), (0.0, 0.0)]
fuzzy :  None
pembilang :  0.3584373094170405
penyebut :  0.944394618834081
rata-rata tertimbang :  0.37954188034188036
 
[0.0, 0.4092, 0.4436, 0.4528, 0.6288, 0.6632, 0

In [31]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.890000,4.180000,0.600000,2.508000,2.50,0.200000,0.500,6.68,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.4,8.574000,0.4,0.5,1.714800
1,3.890000,0.797690,0.220561,0.599578,4.83,0.276471,0.966,4.83,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,2.554000,0.4,0.5,0.510800
2,3.890000,0.510000,0.200000,0.102000,0.23,0.200000,0.046,0.74,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.548000,0.4,0.5,0.309600
3,3.890000,4.258745,0.200000,1.549524,10.80,0.600000,6.480,10.80,6.0,0.6,...,0.2,0.4,0.0,0.2,0.0,4.4,17.530000,0.4,0.5,3.506000
4,3.890000,0.559720,0.200000,0.131775,0.08,0.200000,0.016,0.08,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.416000,0.4,0.5,0.283200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,15.375265,0.000000,0.200000,0.000000,0.40,0.200000,0.080,0.89,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.578000,0.4,0.5,0.315600
772,55.570000,0.470000,0.200000,0.094000,0.37,0.200000,0.074,0.00,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.568000,0.4,0.5,0.291355
773,55.570000,0.240000,0.200000,0.000000,0.00,0.200000,0.008,0.28,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.456000,0.4,0.5,0.291200
774,55.570000,0.200000,0.200000,0.040000,0.00,0.200000,0.000,0.20,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.436917,0.4,0.5,0.288000


In [32]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.0,776.000000
mean,19.612130,2.518866,0.348187,1.263414,8.537929,0.308566,5.997980,13.056525,5.065097,0.223266,...,0.205155,0.508701,0.096649,0.216534,0.045361,2.695416,15.465410,0.552290,0.5,6.515103
std,23.809602,5.625186,0.172998,3.430503,21.591608,0.167353,30.002743,51.053036,0.255252,0.079001,...,0.033300,0.273892,0.522712,0.065949,0.287016,0.852679,38.834369,0.432006,0.0,37.090774
min,1.850000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.000000,1.400000,0.000000,0.5,0.280000
25%,2.370000,0.407871,0.200000,0.000000,0.000000,0.200000,0.000000,0.607500,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.000000,1.578000,0.400000,0.5,0.315600
50%,3.890000,0.945000,0.200000,0.148000,0.230000,0.200000,0.034000,1.370000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.200000,2.200000,0.400000,0.5,0.552200
75%,55.570000,2.590000,0.600000,1.321500,2.477500,0.400000,0.753000,7.615000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,3.201333,10.591000,0.400000,0.5,3.267654
max,59.850000,85.000000,0.600000,51.000000,144.390000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,0.600000,2.400000,9.000000,0.600000,5.400000,5.200000,583.688000,5.800000,0.5,961.514400


# Klasifikasi

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [34]:
x = dummy.values
y = df.iloc[:, -1].values

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [36]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [37]:
classifier = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [  7  63]]
0.9639175257731959


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [38]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [  8  62]]
0.9587628865979382


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [39]:
classifier = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [  8  62]]
0.9587628865979382


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [40]:
classifier = KNeighborsClassifier(n_neighbors=9, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [ 10  60]]
0.9484536082474226


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
